In [ ]:
import pandas as pd
import json
import numpy as np

In [ ]:
df = pd.read_pickle('../data/pickle/dep.pickle')

In [ ]:
prereq_for = df.prerequisite_for.apply(lambda x: " ".join(list(x)))

In [ ]:
import re
regex = re.compile("([A-Za-z]{2,6}[-\s]*\d{3}(?:\s*\([A-Za-z0-9]+\))?)")

In [ ]:
prereq_matches = prereq_for.apply(lambda x: regex.findall(x))
prereq = prereq_matches[prereq_matches.apply(lambda x: len(x) > 0)]
prereq

In [ ]:
required = df.required.apply(lambda x: " ".join(list(x)))

req_matches = required.apply(lambda x: regex.findall(x))
req = req_matches[req_matches.apply(lambda x: len(x) > 0)]
req

In [ ]:
recommended = df.recommended.apply(lambda x: " ".join(list(x)))

rec_matches = recommended.apply(lambda x: regex.findall(x))
rec_matches[rec_matches.apply(lambda x: len(x) > 0)]

In [ ]:
#Idea : concat together required and prereq_for
#first, inverse the required into prerequisite
inv_req = {}
for k, v in req.items():
    for e in v:
        _e = e.upper()
        inv_req[_e] = inv_req.get(_e, [])
        inv_req[_e].append(k.upper())

        
print(len(inv_req))
inv_req

In [ ]:
for k,v in prereq.items():
    _k = k.upper()
    inv_req[_k] = inv_req.get(_k,[])
    for elem in v:
        inv_req[_k].append(elem.upper())

In [ ]:
#merged version
print(len(inv_req))
inv_req

In [ ]:
for key, value in inv_req.items():
     inv_req[key] = set(inv_req[key])

In [ ]:
print(len(inv_req))
inv_req

In [ ]:
def dict_to_df(x):
    res = pd.DataFrame.from_dict(x, orient='index')
    res['prerequisite_for'] = res.apply(list, axis=1)
    res = res['prerequisite_for'].map(lambda x: list(filter(lambda a: a!= None, x))).reset_index().rename(columns={'index':'course_code'})
    return res

In [ ]:
inv_df = dict_to_df(inv_req)
inv_df.tail()

In [ ]:
with open(f'../data/epfl_prereq_byhand.json') as file:
        by_hand = json.load(file)
by_hand = dict_to_df(by_hand)
by_hand.head()

In [ ]:
print(inv_df['course_code'].size)
print(by_hand['course_code'].size)

In [ ]:
def merge(df1, df2):
    merged = df1.merge(df2, on='course_code', how='outer')
    merged = merged.apply(lambda x: x.apply(lambda x: [] if x is np.nan else x))
    merged['prerequisite_for'] = merged['prerequisite_for_x'] + merged['prerequisite_for_y']
    merged = merged[['course_code','prerequisite_for']]
    merged['prerequisite_for'] = merged['prerequisite_for'].apply(lambda x: sorted(list(set(x))))
    return merged.set_index('course_code')

In [ ]:
merged = merge(inv_df, by_hand)
merged.head()

In [ ]:
merged = merged.sort_index()
merged.head()

In [ ]:
merged['prerequisite_for'].to_json(path_or_buf='../data/merged_prerequisites.json', orient='index')